# Importandos todas as Bibliotecas necessárias.

In [13]:
import pandas as pd
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx2pdf import convert
import time
from num2words import num2words
import re
import os
from unidecode import unidecode
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pyautogui
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

# BuscarDados

Classe que realiza a busca de dados na planilha do Excel e retorna um objeto:<br>

 * Possui os seguinte parâmetros: 
   * busca = o dado que você está procurando na tabela, a busca não é *case sensitive*;
   * aba = pode ser fornecida a aba onde está o dado, caso contrário será feita a busca em todas as abas e retornará a primeira linha com o dado;
   * arquivo = pode ser fornecido o nome do arquivo excel, caso contrário usará o arquivo padrão especificado quando da criação da classe;
   <br><br>
 
 * Arquivo do Excel com uma tabela por aba;

 * Será criado um atributo dicionário, com as chaves (*colunas*) e os valores respectivos;
 
 * Se a célula for vazia, a mesma será excluído do dicionário;
 
 * Importante analisar a ordem das abas, abas com valores únicos devem ser as primeiras da planilha;

 * Se o dado procurado não está no arquivo excel, retornará o erro: **'O valor não foi encontrado nas abas do arquivo'**

In [14]:
class BuscarDados:

    def __init__(self, busca, aba=None, arquivo='01_Cadastro.xlsm'):
        
        banco_de_dados = (
            pd.read_excel(arquivo, aba, dtype=str) if aba
            else pd.concat(pd.read_excel(arquivo, aba, dtype=str), ignore_index=True)
        )
        
        banco_de_dados = banco_de_dados.applymap(lambda texto: texto.lower() if isinstance(texto, str) else texto)
        busca = str(busca).lower()

        try:
            self.dicionario = banco_de_dados[banco_de_dados.eq(busca).any(axis=1)].iloc[0].dropna().to_dict()
        except IndexError:
            raise ValueError('O valor não foi encontrado nas abas do arquivo')
        
        self.dicionario = {chave.lower(): valor for chave, valor in self.dicionario.items()}

In [37]:
class FormatarTexto:
    
    def __init__(self, dicionario):
        
        maiusculas = 'nome','uf','expedidor_rg','administrador','comarca_uf'
        minusculas = 'de','com','da','das','do','dos','na','nas','no','nos','o','os','a','as','uma','umas','um','uns'
        inicial_maiuscula = 'logradouro','complemento','bairro','cidade','vara','comarca'

        for chave, valor in dicionario.items():
            
            if chave in maiusculas:
                dicionario[chave] = valor.upper()
            
            elif chave in inicial_maiuscula:
                palavras = valor.split()
                palavras_atualizadas = (
                    palavra.title() if palavra.lower() not in minusculas else palavra
                    for palavra in palavras
                )
                dicionario[chave] = ' '.join(palavras_atualizadas)

In [16]:
class AgruparEndereco:
    def __init__(self, dicionario):
        
        try:
            n = f"{dicionario['numero']} - {dicionario ['complemento']}" if 'complemento' in dicionario else dicionario['numero']
            endereco = f"{dicionario['logradouro']}, nº {n}, Bairro {dicionario['bairro']}, CEP {dicionario['cep']}, {dicionario['cidade']}/{dicionario['uf']}"
            dicionario['endereco'] = endereco
        except KeyError as e:
            raise ValueError(f'O dicionário não possui a chave {e.args[0]} para construir o endereço')

In [17]:
class QualificarParte:
    
    def __init__(self, dicionario):
        
        if 'cpf' in dicionario:
            qualificacao = (
                f"{dicionario['nome']}, {dicionario['nacionalidade']}, {dicionario['estado_civil']}, {dicionario['profissao']}, RG n° {dicionario['rg']} "
                f"{dicionario['expedidor_rg']}, CPF n° {dicionario['cpf']}, e-mail: {dicionario['email']}, com endereço à {dicionario['endereco']}"
            )

        elif 'tag' in dicionario and dicionario['tag'] == 'condominio':
            qualificacao = (
                f"{dicionario['nome']}, inscrito no CNPJ n° {dicionario['cnpj']}, com endereço à {dicionario['endereco']}, "
                f"devidamente representado neste ato pelo seu síndico(a) {dicionario['administrador']}"
            )
        
        else:
            qualificacao = (
                f"{dicionario['nome']}, pessoa jurídica inscrita no CNPJ n° {dicionario['cnpj']}, com endereço à {dicionario['endereco']},"
                f" devidamente representada neste ato pelo(a) sócio(a) {dicionario['administrador']}"
            )
        
        dicionario['qualificacao'] = qualificacao

In [18]:
class CriarAtributos:

    def __init__(self, objeto):
        
        for chave, valor in objeto.dicionario.items():
            setattr(objeto, chave.lower(), valor)

In [19]:
class DividirValor:
    
    def __init__(self, dicionario, separador):
        dicionario.update({chave: valor.split(separador) for chave, valor in dicionario.items() if separador in valor})

In [20]:
class ParearAtributos:
    
    def __init__(self, dicionario, par_1, par_2):
        
        CriarLista(dicionario, par_1)
        CriarLista(dicionario, par_2)
        dicionario.update(dict(zip(dicionario[par_1], dicionario[par_2])))

In [21]:
class CriarLista:
    
    def __init__(self, dicionario, valor):
        if valor in dicionario:
            dicionario[valor] = [dicionario[valor]] if isinstance(dicionario[valor], str) else dicionario[valor]

In [30]:
class adicionar_dados_do_processo:
    
    def __init__(self, dicionario):

        if not str(dicionario['id_processos']).isalpha():
            dicionario.update(BuscarDados(dicionario['id_processos'], 'Processos').dicionario)
        
        else: 
            dicionario['VARA'] = 'Vara Cível'
            dicionario['COMARCA'] = 'S. J. Rio Preto/SP'        
        
        FormatarTexto(dicionario)

In [23]:
class CriarPessoa:

    def __init__(self, dicionario):
        
        FormatarTexto(dicionario)
        AgruparEndereco(dicionario)
        QualificarParte(dicionario)

In [39]:
class CriarPartes:
    
    def __init__(self, dicionario):
        
        for parte in dicionario['partes']:
            for i, pessoa in enumerate(dicionario[parte], start=1):
                prefixo = f'[{i}]{parte}_'
                
                dicionario_pessoa = BuscarDados(pessoa).dicionario
                CriarPessoa(dicionario_pessoa)

                for chave, valor in dicionario_pessoa.items():
                    dicionario[prefixo + chave] = valor

                if len(dicionario['partes']) == 1:
                    dicionario[parte + '_qualificacao'] = dicionario[f'{prefixo}qualificacao']                    
                elif i == 1:
                    dicionario[parte + '_qualificacao'] = f'[{i}] ' + dicionario[f'{prefixo}qualificacao']
                elif i == len(dicionario['partes']):
                    dicionario[parte + '_qualificacao'] += ' e ' + f'[{i}] ' + dicionario[f'{prefixo}qualificacao']
                else:
                    dicionario[parte + '_qualificacao'] += ', ' + f'[{i}] ' + dicionario[f'{prefixo}qualificacao']

In [25]:
class CriarNomeArquivo:

     def __init__(self, dicionario):      
        
        peca = dicionario['peca'] + '_'

        if isinstance(dicionario['partes'], str):
            parte = dicionario['pessoas']        
        else:
            parte = dicionario['pessoas'][0]
        
        dicionario['arquivo'] = unidecode((peca + parte).replace(' ','-'))

In [26]:
class CriarDocumento:

    def __init__(self, id_documento):

        self.dicionario = BuscarDados(id_documento, 'Peticoes').dicionario
        self.dicionario.update(BuscarDados(self.dicionario['peca'], 'Modelos').dicionario)
        
        DividirValor(self.dicionario, ';')
        if 'atributos' in self.dicionario:
            ParearAtributos(self.dicionario, 'atributos', 'valores')
        ParearAtributos(self.dicionario, 'partes', 'pessoas')
        
        DividirValor(self.dicionario, '/')        

        for parte in self.dicionario['partes']:
            CriarLista(self.dicionario, parte)
        for pessoa in self.dicionario['pessoas']:
            CriarLista(self.dicionario, pessoa)
        
        adicionar_dados_do_processo(self.dicionario)
        CriarPartes(self.dicionario)
        CriarNomeArquivo(self.dicionario)

        #CriarAtributos(self)

Salva o arquivo em docx e pdf:

* Arquivos devem ser escolhidos pelo id

* Substitui os placeholders(chaves) de um arquivo docx pelos valores contidos nas chaves do dicionário criado com 'CriarDocumento';<br>

* Formata o nome do arquivo de acordo com o padronizado;

* Na sequência, salva em um arquivo docx em uma pasta já especificada e converte também em pdf.

In [15]:
class SalvarDocxPdf:

    def __init__(self, id_documento):
        
        dicionario = CriarDocumento(id_documento).dicionario
        documento = Document(f"01_Modelos\{dicionario['peca']}.docx")
        self._substituir_texto(dicionario, documento)      
        
        documento.save(f"02_Documentos-prontos\{dicionario['arquivo']}.docx")
        convert(f"02_Documentos-prontos\{dicionario['arquivo']}.docx", f"02_Documentos-prontos\{dicionario['arquivo']}.pdf")

    def _substituir_texto(self, dicionario, docx):

        def numero_por_extenso(paragrafo):
            
            numero_entre_parenteses = r"\((.*?)\)"
            resultado_busca_numero = re.search(numero_entre_parenteses, paragrafo.text)
            if resultado_busca_numero:
                valor_encontrado = resultado_busca_numero.group(1)
                valor_decimal = valor_encontrado.replace('.', '').replace(',', '.')
                try:
                    valor_por_extenso = '(' + num2words(float(valor_decimal), lang='pt_BR', to='currency') + ')'
                    paragrafo.text = paragrafo.text.replace(resultado_busca_numero.group(0), valor_por_extenso)
                except ValueError:
                    pass

        dicionario_de_substituicao = {'x' + chave.upper() + 'x': str(valor) for chave, valor in dicionario.items()}
        
        for paragrafo in docx.paragraphs:
            for chave in dicionario_de_substituicao:

                if not isinstance(chave, list):
                    if chave in paragrafo.text:
                        paragrafo.text = paragrafo.text.replace(chave, dicionario_de_substituicao[chave])
                numero_por_extenso(paragrafo)

___
